In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Convolution2D, Dropout, Dense, Flatten, BatchNormalization
from keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import Adam

import pandas as pd
import numpy as np


Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
print(X_train.shape,y_train.shape,y_test.shape,y_test.shape)

(60000, 28, 28) (60000,) (10000,) (10000,)


In [4]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [5]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [6]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape =(28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size = 4, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))


In [8]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train_cat, batch_size=32, epochs=2, verbose=1, validation_split=0.3)

Train on 42000 samples, validate on 18000 samples
Epoch 1/2
42000/42000 [==============================] - 890s 21ms/sample - loss: 0.2197 - accuracy: 0.9332 - val_loss: 0.0659 - val_accuracy: 0.9808
Epoch 2/2
42000/42000 [==============================] - 527s 13ms/sample - loss: 0.0783 - accuracy: 0.9764 - val_loss: 0.0525 - val_accuracy: 0.9842


In [10]:
model.evaluate(X_test, y_test_cat)

10000/10000 [==============================] - 12s 1ms/sample - loss: 0.0435 - accuracy: 0.9848


[0.043512688274774704, 0.9848]

In [11]:
import matplotlib as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

from numpy import argmax
from tkinter import *
import tkinter as tk
import math
from PIL import Image, ImageDraw

In [ ]:
white = (255, 255, 255)
black = (0, 0, 0)
window = Tk()
 
window.title("Handwritten digits")
 
window.geometry('500x500')
 
lbl = Label(window, text="Write digit",font=('Arial Bold',15))
 
lbl.grid(column=3, row=0)
 
canvas_width = 120
canvas_height = 120
image1 = Image.new("RGB", (canvas_width, canvas_height),white)
draw = ImageDraw.Draw(image1)
counter=0
xpoints=[]
ypoints=[]
x2points=[]
y2points=[]
global predictions
predictions = []
number1 = []
digits=0

def paint( event ):
    x1, y1 = ( event.x - 4 ), ( event.y - 4 )
    x2, y2 = ( event.x + 4 ), ( event.y + 4 )
    w.create_oval( x1, y1, x2, y2, fill = 'black' )
    xpoints.append(x1)
    ypoints.append(y1)
    x2points.append(x2) 
    y2points.append(y2)    
    
def imagen ():
    global counter
    global xpoints
    global ypoints    
    global x2points
    global y2points
    counter=counter+1

    image1 = Image.new("RGB", (canvas_width, canvas_height),black)
    draw = ImageDraw.Draw(image1) 

    elementos=len(xpoints)
    
    

    for p in range (elementos):
        x=xpoints[p]
        y=ypoints[p]
        x2=x2points[p]
        y2=y2points[p] 
        draw.ellipse((x,y,x2,y2),'white')
        w.create_oval( x-4, y-4, x2+4, y2+4,outline='gray85', fill = 'gray85' )

    size=(28,28)
    image1 = image1.resize(size)

    
    image1 = image1.convert('L')
    image1 = np.array(image1)
    image1 = image1.reshape(-1, 28, 28, 1)
    image1 = image1.astype('float32')
    image1 /= 255.0

    
    predictions.append(argmax(model.predict(image1)))
    lbl2 = Label(window, text=predictions[counter-1],font=('Arial Bold',20))
    lbl2.grid(column=3, row=10)
    

    xpoints=[]
    ypoints=[]
    x2points=[]
    y2points=[] 


w = Canvas(window, 
           width=canvas_width, 
           height=canvas_height,bg='gray85')
w.grid(column=3,row=2)

def delete ():
    global counter
    counter = counter-1
    del predictions[counter]
    w1 = Canvas(window, 
           width=200, 
           height=20,bg='gray95')
    w1.grid(column=3,row=10)
    
w.bind( "<B1-Motion>", paint )
button = tk.Button(window, text='Save image', width=25, command=imagen)
button.grid(column=3,row=3)

button6 = tk.Button(window, text='Click here if the number is not correct', width=35, command=delete)
button6.grid(column=3,row=12)


window.mainloop()